In [ ]:
%%capture
# !pip install accelerate==0.21.0 \
#   bitsandbytes==0.40.2 \
#   peft==0.5.0 \
#   transformers==4.34.0 \
#   sentencepiece \
#     langchain \
#      faiss-cpu \
#     sentence-transformers

# !pip install -U bitsandbytes 
# !pip install -U transformers 
# !pip install -U peft 

In [1]:
%%capture
!pip install "numpy>=1.12.1"
!pip install "scipy>=1.0.1"
!pip install "torch>=2.0.0"
# !pip install transformers==4.34.0
!pip install transformers==4.35.2
!pip install accelerate==0.23.0
!pip install bitsandbytes==0.41.0
!pip install peft==0.5.0
!pip install pillow==10.0.1
# !pip install llama-cpp-python
# !pip install datasets
!pip install zstandard
!pip install jsonlines
!pip install sentencepiece
# !pip install fire
!pip install datasketch==1.5.9
!pip install nltk==3.8.1
# !pip install scikit-learn==1.3.0
!pip install pytest

In [2]:
import gc
gc.collect()

0

In [3]:
import transformers
print(transformers.__version__)
# 4.35.2

4.35.2


In [1]:
import torch
# from langchain.document_loaders import DataFrameLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS
import pandas as pd
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch.nn.functional as F

from typing import Any, List, Mapping, Optional

# from langchain.callbacks.manager import CallbackManagerForLLMRun
# from langchain.llms.base import LLM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


- [PEFT код](https://github.com/huggingface/peft/blob/main/src/peft/peft_model.py#L264)
- [Модель Mistral_saiga_7b ток в точность fp16](https://huggingface.co/Gaivoronsky/Mistral-7B-Saiga)

In [2]:

for d in {1:2, 3:4}.items():
    print()

In [17]:
# MODEL_NAME = "IlyaGusev/saiga_mistral_7b_lora"
# MODEL_NAME = "Gaivoronsky/Mistral-7B-Saiga"
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
# DEFAULT_SYSTEM_PROMPT = "Ты извлекаешь информацию из текста"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()



def generate(model, tokenizer, prompt, generation_config):
    print()
    data = tokenizer(prompt,
                     return_tensors="pt",
                     add_special_tokens=False,
    #                      padding=True,
    #                     truncation=True
                    )
    #print(data)
    data = {k: v.to(device) for k, v in data.items()}
    
    output_ids = model.generate(
        **data,
        generation_config = generation_config
    #         remove_invalid_values = True
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=False)
    return output.strip()



In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [5]:
import time

st_time = time.time()
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit = True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True,
    device_map="auto"
)


model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
print(f'Прошло времени {time.time() - st_time}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}

Прошло времени 114.5029125213623


In [6]:
config

LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='TheBloke/Llama-2-7B-fp16', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj'], lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [7]:
%ls

In [8]:
print_trainable_parameters(model)

trainable params: 16777216 || all params: 6755192832 || trainable%: 0.24836028248556738


In [9]:
# generation_config.__delattr__("top_p")
# generation_config

In [10]:
# generation_config.__delattr__("top_p")
# generation_config

In [11]:
print(tokenizer.decode([0]))
print(tokenizer.unk_token)

<unk>
<unk>


In [12]:
# Изменение конфигурации
generation_config.temperature = 0.3
generation_config.no_repeat_ngram_size = 15
generation_config.do_sample = True
generation_config.top_k = 40
generation_config.top_p = 0.9
# generation_config.num_beams=1
generation_config.unk_token = 0
print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.3,
  "top_k": 40,
  "top_p": 0.9,
  "unk_token": 0
}



In [18]:
%%time
def get_message(inputs):
    conversation = Conversation()
    conversation.add_user_message(inputs)
    prompt = conversation.get_prompt(tokenizer)
    print('Промт', '\n', '*'*100)
    print(prompt)
    print('*'*100)
    output = generate(model, tokenizer, prompt, generation_config)
    return output

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10.5 µs


In [ ]:
%%time
p = """

"""
get_message([p])

In [20]:
get_message('сколько в среднем весит человек')

Промт 
 ****************************************************************************************************
<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user
сколько в среднем весит человек</s><s>bot
****************************************************************************************************



'В среднем мужчины на Земле весят 82,4 кг (181 фунта), а женщины - 69,5 кг (153 фунтов). Однако эти цифры могут сильно отличаться от страны до страны или региона. Например, в США средний вес мужчин составляет около 90-97 килограммов (около 198-212 фунтов) при росте 175 см (5\'9"), тогда как у женщин это 75-80 кг (165-176 фунтов) при росте 160 см (5’3"). В Японии же средняя масса взрослых людей составляет всего 58,5 кг (129 фунты) для мужчин и 52,2 кг (115 фунтов) для женщин.</s>'

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.decode([1]))
print(tokenizer.decode([2]))